In [1]:
import numpy as np
from numpy.testing import assert_almost_equal

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
from lsst.meas.mosaic.updateExposure import applyMosaicResultsCatalog, getWcs


In [2]:
frame = 1
dispL2 = afwDisplay.getDisplay(frame)
dispL2.setMaskTransparency(75)

In [3]:
rootDirL2 = "/tigress/HSC/HSC/rerun/lauren/LSST/DM-9862/cosmos_noJunk/orig/"

# make a butler
butlerL2 = dafPersist.Butler(rootDirL2)

In [4]:
visit = 1228
ccdList = [49, 50, 100, 101, 102, 103]
filter = "HSC-I"
tract = 0
srcIndex = 100

knownRa = [150.291755265, 149.915585934, 150.761245879, 149.406088728, 150.77650489, 149.407912134]
knownDec = [2.20870255223, 2.16278669124, 1.84687842649, 1.84855672765, 2.55532853412, 2.55168410547]
knownX = [853.076542845, 218.0, 990.451759767, 109.935039499, 694.488613548, 1873.73688523]
knownY = [186.383389529, 210.0, 289.913434466, 256.140451952, 298.552189014, 275.660772235]
knownModelFlux = [1477.6487901, 3948.61992604, 5022.95936829, 9588.90750291, 3467.05160473, 72949.3415395]

for i, ccd in enumerate(ccdList):
    dataId = {"visit": visit, "ccd": ccd, "tract": tract}
    srcCatL2 = butlerL2.get("src", dataId)
    s2 = srcCatL2[srcIndex]
    s2.getCoord()
    dataRef2 = butlerL2.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
    result2 = applyMosaicResultsCatalog(dataRef2, srcCatL2, True)
    cat2 = result2.catalog
    s = cat2[srcIndex]
    # print float(s.getRa().asDegrees()) #, s.getDec(), s.getX(), s.getY(), s.getModelFlux()
    assert_almost_equal(float(s.getRa().asDegrees()), knownRa[i], decimal=9)
    assert_almost_equal(float(s.getDec().asDegrees()), knownDec[i], decimal=9)
    assert_almost_equal(s.getX(), knownX[i], decimal=8)
    assert_almost_equal(s.getY(), knownY[i], decimal=8)
    assert_almost_equal(s.getModelFlux(), knownModelFlux[i], decimal=7)

In [5]:
wcs = getWcs(dataRef2)
print wcs

<lsst.afw.image.imageLib.Wcs; proxy of <Swig Object of type 'std::shared_ptr< lsst::afw::image::Wcs > *' at 0x2b6f04d75a20> >


In [6]:
expL1 = butlerL1.get("calexp", dataId)
expL2 = butlerL2.get("calexp", dataId)

NameError: name 'butlerL1' is not defined

In [9]:
dispL1.mtv(expL1.getMaskedImage(), title="LSST image")
dispL2.mtv(expL2.getMaskedImage(), title="LSST image")

In [5]:
srcCatL1 = butlerL1.get("src", dataId)
srcCatL2 = butlerL2.get("src", dataId)

In [6]:
s1 = srcCatL1[10]
s2 = srcCatL2[10]

In [7]:
s1.getCoord()

IcrsCoord(149.90675487730076*afwGeom.degrees, 2.2354969394502535*afwGeom.degrees)

In [8]:
s2.getCoord()

IcrsCoord(149.90675487730076*afwGeom.degrees, 2.2354969394502535*afwGeom.degrees)

In [9]:
dataRef1 = butlerL1.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
result1 = applyMosaicResultsCatalog(dataRef1, srcCatL1, True)
dataRef2 = butlerL2.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
result2 = applyMosaicResultsCatalog(dataRef2, srcCatL2, True)

HERE!!...
1: ffp.wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
2: ffp.wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]
2: wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
HERE!!...
1: ffp.wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]
2: ffp.wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
2: wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]


In [10]:
cat1 = result1.catalog
cat2 = result2.catalog

In [11]:
cat1[10].getCoord()

IcrsCoord(149.9068054393456*afwGeom.degrees, 2.2354459115086627*afwGeom.degrees)

In [12]:
cat2[10].getCoord()

IcrsCoord(150.10030301657136*afwGeom.degrees, 2.165451941318979*afwGeom.degrees)

In [29]:
dispL1.mtv(expL1.getMaskedImage(), title="LSST image")

In [30]:
dispL2.mtv(expL2.getMaskedImage(), title="LSST image")

In [15]:
wcs = getWcs(dataRef1)

In [16]:
wcs.getCDMatrix()

array([[ -7.79102403e-10,   4.68748816e-05],
       [  4.68612723e-05,   8.23129204e-09]])